### RGPD exclusion process
- Take the 2024 beneficiaries data into a dataframe
- Exclude from this 2024 beneficiaries dataframe the people that are inside
  - The list of codes to exclude
  - The list of emails to exclude
- Create a CSV file with the excluded rgpd lists

In [ ]:
import csv
import pandas as pd
import os
import json

from dotenv import load_dotenv

load_dotenv()

pathfile_beneficiaries_2024 = os.environ['DB_CNAF_MSA_2024']
pathfile_rgpd_emails_to_exclude =  os.environ['RGPD_EMAILS_TO_EXCLUDE_PATHFILE']
pathfile_rgpd_codes_to_exclude =  os.environ['RGPD_CODES_TO_EXCLUDE_PATHFILE']

output_pathfile_filtered_beneficiaries = os.environ['DB_CNAF_MSA_2024_WITHOUT_RGPD']

df_2024 = pd.read_csv(pathfile_beneficiaries_2024, sep=';', encoding='utf-8', quoting=csv.QUOTE_ALL)
df_rgpd_emails_to_exclude = pd.read_csv(pathfile_rgpd_emails_to_exclude)
df_rgpd_codes_to_exclude = pd.read_csv(pathfile_rgpd_codes_to_exclude)

In [ ]:
# Extract emails from json
df_json_allocataire = pd.json_normalize(df_2024['allocataire'].apply(json.loads)).add_prefix('allocataire-')
df_2024_unwrapped = pd.merge(df_2024, df_json_allocataire['allocataire-courriel'], left_index=True, right_index=True)

In [ ]:
emails_to_exclude_mask = ~df_2024_unwrapped['allocataire-courriel'].isin(df_rgpd_emails_to_exclude['email'])
codes_to_exclude_mask = ~df_2024_unwrapped['id'].isin(df_rgpd_codes_to_exclude['id'])

In [ ]:
df_with_excluded_data_2024 = df_2024_unwrapped[emails_to_exclude_mask & codes_to_exclude_mask]

In [ ]:
assert(len(df_2024_unwrapped) > len(df_with_excluded_data_2024))

In [ ]:
df_with_excluded_data_2024.drop(columns=['id', 'allocataire-courriel'], inplace=True)

In [ ]:
df_with_excluded_data_2024.to_csv(output_pathfile_filtered_beneficiaries, sep=';', index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)